# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится.

Построим модель с максимально большим значением *accuracy*. 

Наметим план:

1. Выгрузка и обзор данных.
2. Разработка модели.
 - Разбивка на выборки.
 - Определение признаков.
 - Разработка моделей:
     * Дерево решений
     * Лес деревьев
     * Логистическая регрессия
 - Проверка модели
 - Проверка модели на адекватность.

## Выгрузка и обзор данных.

In [1]:
#Импортирую необходимые модули и библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from joblib import dump

In [2]:
#Открываю файл с данными и сохраняю в переменную df
df = pd.read_csv('/Users/mariaspartalan/Desktop/готовые проекты/5 Мегалайн/users_behavior.csv')

In [3]:
#Какие данные нам даны
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Итак, у нас есть датафрейм в котором есть информация о 3124 обьектах.

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. 

Известно:

* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

    Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».


    В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. 

Предобратка не понадобиться, она уже сделана.




## Разработка модели.
### Разбейте данные на выборки

In [4]:
#Разделяю данные на 3 подборки в 2 два этапа

df_train, df_valid_test = train_test_split(df, test_size=0.4, random_state=12345, stratify=df['is_ultra'])

In [6]:
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=12345, stratify=df_valid_test['is_ultra'])

In [7]:
print('Размеры выборок:\nОбщая {}\nТренировачная{}\nВалидационная{}\nТестовая{}'
      .format(df.shape, df_train.shape, df_valid.shape, df_test.shape))

Размеры выборок:
Общая (3214, 5)
Тренировачная(1928, 5)
Валидационная(643, 5)
Тестовая(643, 5)


________________
Промежуточный вывод

В этой части работы я разделила весь датафрейм на 3 подборки:
-тренировочную,
-валидационную,
-тестовую.

Даные разделены в пропорциях 3:1:1.

###  Определение признаков:


   Нужно построить модель для задачи классификации, которая выберет подходящий тариф. То есть целевой признак это is_ultra, остальные столбцы признаки на основании данных которых будет работать наша модель. Создадим датафреймы с данными для кадого набора

In [8]:
def new_df (data):
    features = data.drop(['is_ultra'], axis=1)
    target = data['is_ultra']
    return features, target

features_train, target_train = new_df(df_train)
features_valid, target_valid = new_df(df_valid)
features_test, target_test= new_df(df_test)

In [9]:
#Проверяю рандомно, что вышло
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_valid.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)


Рассмотрим разные модели для предсказания целевого признака. Расчитаем количество ошибок для каждого.

#### Дерево решений.

Создаю цикл, который создаcт 5 деревьев с различной глубиной, проверит их точность.

In [9]:
best_tree = None
best_result_tree = 0

for depth in range(1,6):
    tree = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    tree.fit(features_train, target_train)
    predictions_valid_tree = tree.predict(features_valid)
    result_tree = accuracy_score(target_valid, predictions_valid_tree)
    print('При глубине дерева {} точность:{}'.format(depth, result_tree))
    if result_tree > best_result_tree:
        best_tree = tree
        best_result_tree = result_tree
print()        
print('Точность лучшей модели:', best_result_tree)       

При глубине дерева 1 точность:0.7402799377916018
При глубине дерева 2 точность:0.7729393468118196
При глубине дерева 3 точность:0.7776049766718507
При глубине дерева 4 точность:0.7542768273716952
При глубине дерева 5 точность:0.7853810264385692

Точность лучшей модели: 0.7853810264385692


Лучший результат показывает дерево с максимальной глубиной 3. Именно эту модель мы и схораним для дальнейшей работы.

Модели с глубиной 1-2 соoтвественно недообучена, в то время как 4-5 переобучена.

В исходном задании значиение accuracy должно быть от 0.75, значит подойдет любое дерево.

#### Случайный лес.

Рассмотрим подробно все варианты эффективности с количество деревьев от 1 до 10.

In [10]:
best_forest = None
best_result_forest = 0

for n_est in range(1, 11):
    forest = RandomForestClassifier(random_state=12345, n_estimators=n_est)
    forest.fit(features_train, target_train)
    predictions_valid_forest = forest.predict(features_valid)
    result_forest = accuracy_score(target_valid, predictions_valid_forest) 
    print('Модель случайного леса из {} деревьев показывает точность:{}'.format(n_est,  result_forest))
    if result_forest > best_result_forest:
        best_forest = forest
        best_result_forest = result_forest
print()        
print('Точность лучшей модели:', best_result_forest)       

Модель случайного леса из 1 деревьев показывает точность:0.6998444790046656
Модель случайного леса из 2 деревьев показывает точность:0.7433903576982893
Модель случайного леса из 3 деревьев показывает точность:0.7418351477449455
Модель случайного леса из 4 деревьев показывает точность:0.7558320373250389
Модель случайного леса из 5 деревьев показывает точность:0.7527216174183515
Модель случайного леса из 6 деревьев показывает точность:0.7729393468118196
Модель случайного леса из 7 деревьев показывает точность:0.7698289269051322
Модель случайного леса из 8 деревьев показывает точность:0.776049766718507
Модель случайного леса из 9 деревьев показывает точность:0.7698289269051322
Модель случайного леса из 10 деревьев показывает точность:0.7869362363919129

Точность лучшей модели: 0.7869362363919129


В задании требуется подобрать модель со значением accuracy не менее 0.75. Следовательно, случайный лес с количеством деревьев 1, 3 и 5 не подходят.

Для дальнейшей работы сохраним модель с 10 деревьями, так как она показала самую высокую точность.

#### Логистическая регрессия

In [11]:
#Поиграем с параметром с

c_param = [0.01, 1, 10, 100, 100000, 100000000000]

for c in c_param:
    model_lr = LogisticRegression(random_state=12345, C=c)
    model_lr.fit(features_train, target_train)
    predictions_valid_lr = model_lr.predict(features_valid)
    result_lr = accuracy_score(target_valid, predictions_valid_lr)
    print('При с={}, точность логистической регрессии:{}'.format(c,  result_lr))

При с=0.01, точность логистической регрессии:0.7387247278382582
При с=1, точность логистической регрессии:0.7387247278382582
При с=10, точность логистической регрессии:0.7387247278382582
При с=100, точность логистической регрессии:0.7387247278382582
При с=100000, точность логистической регрессии:0.7387247278382582
При с=100000000000, точность логистической регрессии:0.7387247278382582


In [12]:
model_lr = LogisticRegression(random_state=12345, class_weight='balanced')
model_lr.fit(features_train, target_train)
predictions_valid_lr = model_lr.predict(features_valid)
result_lr = accuracy_score(target_valid, predictions_valid_lr)
print('Точность при class_weight=balanced:', result_lr)

Точность при class_weight=balanced: 0.6220839813374806


Параметр C не влияет на точность полученной модели.

Точность меньше необходимый 0.75. Модель логистической регрессии нам не подходит.

__________________

**Промежуточный вывод:**

Для данной задачи самую низку эффективность показала модель логистической регрессии. 

У дерева с максимальной губиной 3 средние результаты, но в принципе подходящие по заявленному в задаче условию, что точность должна быть не менее 0.75.

Самый лучший результат показала модель лес случайных деревьев. Accuracy=0.7869362363919129

Вообще обычно у дерева ниже эффективность, но быстрее скорость. В то время, как лес более точен, но затранее в исполнении. 


### Проверьте модели на тестовой выборке

In [13]:
#Проверяю модель на тестововй выборке

predictions_test_forest = best_forest.predict(features_test)

#Расчитываю точность предсказаний тестовой выборки

accuracy_test_forest = accuracy_score(predictions_test_forest, target_test)

#Выбираю лучшую модель

print('Точность леса:', accuracy_test_forest)


Точность леса: 0.7931570762052877


Лучшую точность показывает модель лес случаный деревьев, с количеством счетчиков 10.


### Проверка модели на адекватность

Когда мы говорим о проверке на вменяемость, мы имеем ввиду то, что наша обученная модель работает лучше, чем что-то простое. В случае с задачей классификации чем-то простым может быть, например, случайное угадывание класса или, как в нашем случае, всегда предсказывание большего класса. Если метрика на нашей модели хуже, чем на простой, то очевидно наша модель не годится. В качестве метрики в этой задаче мы используем accuracy - долю правильных ответов. Если мы будем на всех элементах предсказывать 0, то accuracy будет равно доле нулей в столбце is_ultra. То есть суть проверки на вменяемость в нашем случае - это убедиться, что accuracy нашей модели больше, чем доля нулей в is_ultra.

In [14]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
dummy = DummyClassifier(strategy='most_frequent', random_state=42).fit(features_train, target_train)
print('Accuracy Dummy-модели на тестовой выборке:', accuracy_score(target_test, dummy.predict(features_test)))

Accuracy Dummy-модели на тестовой выборке: 0.6936236391912908


Сравнив результат случайного классификатора и результат модели-леса, делаю вывод: 
Модель лес адекватна и хорошо работает.

**ОБЩИЙ ВЫВОД:**

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

Мы выгрузили данные. Предобработка данных не понадобится.

Далее разбили данные на выборки и выделили целевой признак.

Разработали несколько моделей и рассчитали accuracy для каждой.

     * Дерево решений - Точность лучшей модели: 0.7853810264385692
     * Лес деревьев - Точность лучшей модели: 0.7869362363919129
     * Логистическая регрессия - Точность логистической регрессии:0.7387247278382582
Лучший результат показала модель случайного леса.

При проверке на тестовой выборке - Точность леса: 0.7931570762052877

Проверили модели на адекватность. Сравнили точность модели с простейшим алгоритмом предсказания признака с нашей. Точность простейшего классификатора оказалась значительно меньше. Следовательно модель адекватна.